## Intervertebral Disc Segmentation Inference

* Total Case : 000 Case
* Dataset come from Multiple Hospitals 
* Base Network : U-Net 2D
* Two Dataset will be used. (non-image processed vs pixel space resampling)
* Future Test Network : U-Net 3D, Deeplab V3, ETC...
* Using Keras with backend Tensorflow
* Project Purpose : Segmentation performance comparison of various algorithms and preprocessing methods.
* Performance Metrics : Dice score, IoU(Intersection over Union)

## Whole Import

In [ ]:
from time import time
import os
import numpy as np
import cv2
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Activation
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from sklearn.utils import shuffle
from tensorflow.keras import models

# Inference using CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Intervertebral Disc Segmentation
### Image Dataset Loader (2D)

* Dataset Loader by using opencv. Loader by using keras ImageDataGenerator will be worked soon.
* As Preprocessing finished, dataset will be saved as Numpy Compressed format (npz)


In [ ]:
test_loaded = np.load('./Dataset/testset_256.npz')

# best_model_path = './models/Unet-256_256_32-Nadam_1e-4-diceloss-relu-he-batch_32-epoch.01.hdf5'
# test_loaded = np.load('./Dataset/testset_256.npz')
# print(validation_input.shape)
# validation_input = np.expand_dims(test_loaded['dataset'][0,:,:,:], axis=0)

validation_input = test_loaded['dataset']
validation_label = test_loaded['label']

print(validation_input.shape, validation_label.shape)



## Main

In [ ]:
## Custom callback functions
# IoU
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, [1, 2, 3]) + K.sum(y_pred, [1, 2, 3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


# Dice score
def dice_coef(y_true, y_pred):
    # print(y_true.shape, y_pred.shape)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)



## inference run function
def run_inference(array, model_path):
    best_model_path = model_path
    validation_input = np.squeeze(array, axis=0) # (576, 576)
    validation_input = np.squeeze(validation_input, axis=-1) # (576, 576)
    org_shape = np.squeeze(validation_input.shape)
    validation_input = cv2.resize(src=validation_input, dsize=(192, 192), interpolation=cv2.INTER_AREA)
    validation_input = (validation_input - np.min(validation_input))/np.max(validation_input) * 255
    validation_input = np.expand_dims(validation_input, axis=0)
    validation_input = np.expand_dims(validation_input, axis=-1)

    new_model = models.load_model(best_model_path, custom_objects={'iou_coef':iou_coef, 'dice_coef':dice_coef, 'dice_coef_loss':dice_coef_loss})
    new_model.summary()
    pred = new_model.predict(validation_input)
    pred_image = pred[:, :, :, 1] * 255
    pred_image = np.squeeze(pred_image, axis=0) # (576, 576)
    pred_image = cv2.resize(src=pred_image, dsize=(org_shape[0], org_shape[1]), interpolation=cv2.INTER_AREA)
    return np.uint8(pred_image)


## run for each file
## And save ground truth file, prediction file, overlapped with original file saving as image file using CV2
model_path = './models/Unet-192_192_32-Nadam_1e-4-diceloss-relu-he-batch_32-epoch.40.hdf5'
for slice in range(validation_input.shape[0]):
    vinput = np.expand_dims(validation_input[slice,:,:,:], axis=0)
    n_ar = (vinput - np.min(vinput)) / np.max(vinput) * 255
    array_image = np.expand_dims(vinput, axis=-1)
    pred = run_inference(array_image, model_path)
    gt_img = validation_label[slice, :, :, 1] * 255
    gt_sp = './res/gt1/' + str(slice) + '.png'
    pr_sp = './res/pr1/' + str(slice) + '.png'
    ov_sp = './res/ov1/' + str(slice) + '.png'
    if not os.path.exists(os.path.dirname(gt_sp)):
        os.makedirs(os.path.dirname(gt_sp))
    if not os.path.exists(os.path.dirname(pr_sp)):
        os.makedirs(os.path.dirname(pr_sp))
    if not os.path.exists(os.path.dirname(ov_sp)):
        os.makedirs(os.path.dirname(ov_sp))

    cv2.imwrite(gt_sp, gt_img)
    cv2.imwrite(pr_sp, pred)

    label = pred
    test_image = np.squeeze(vinput)
    pred_image = label
    pred_image = np.expand_dims(pred_image, axis=0)
    pred_image = np.expand_dims(pred_image, axis=3)
    G = np.zeros([1, 256, 256, 1])
    B = np.zeros([1, 256, 256, 1])
    R = pred_image
    pred_image = np.concatenate((B, G, R), axis=3)
    pred_image = np.squeeze(pred_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = np.expand_dims(test_image, axis=3)
    tR = test_image
    tG = test_image
    tB = test_image
    test_image = np.concatenate((tB, tG, tR), axis=3)
    test_image = np.squeeze(test_image)
    test_image = test_image.astype(float)
    w = 40
    p = 0.0001
    result = cv2.addWeighted(pred_image, float(100 - w) * p, test_image, float(w) * p, 0) * 255
    cv2.imwrite(ov_sp, result)
    print('Ground Truth saved at {}'.format(gt_sp))
    print('Predict Result saved at {}'.format(pr_sp))
    print('Overlapped Img saved at {}'.format(ov_sp))
    print()